In [1]:
!nvidia-smi

Sat Feb  3 16:26:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              31W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn

torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):   #@save
    """如果存在,则返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices=[torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
X=torch.ones(2,3)
X,X.device

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 device(type='cpu'))

In [6]:
Y=torch.rand(2,3,device=try_gpu())
Y

tensor([[0.7348, 0.0247, 0.9136],
        [0.0666, 0.2517, 0.0792]], device='cuda:0')

In [7]:
Z=X.cuda()
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [8]:
Y+Z

tensor([[1.7348, 1.0247, 1.9136],
        [1.0666, 1.2517, 1.0792]], device='cuda:0')

In [9]:
Z.cuda() is Z

True

In [10]:
net=nn.Sequential(nn.Linear(3,1))
net=net.to(device=try_gpu())
net(Z)

tensor([[-0.7689],
        [-0.7689]], device='cuda:0', grad_fn=<AddmmBackward0>)

Exercises

比较CPU与GPU的速度差异

In [11]:
from d2l import torch as d2l

X=torch.rand(10000,10000)
Y=torch.rand(10000,10000)
timer=d2l.Timer()
Z=torch.mm(X,Y)
print(f'cpu: {timer.stop():.5f} sec')

X=X.cuda()
Y=Y.cuda()
timer.start()
Z=torch.mm(X,Y)
print(f'gpu: {timer.stop():.5f} sec')

cpu: 7.31728 sec
gpu: 0.02744 sec


In [12]:
X=torch.rand(10,10)
Y=torch.rand(10,10)
timer.start()
Z=torch.mm(X,Y)
print(f'cpu: {timer.stop():.5f} sec')

X=X.cuda()
Y=Y.cuda()
timer.start()
Z=torch.mm(X,Y)
print(f'gpu: {timer.stop():.5f} sec')

cpu: 0.00046 sec
gpu: 0.00020 sec


In [13]:
!nvidia-smi # 可以使用nvidia-smi命令查看显存使用情况(如当前显存占用1464MiB)

Sat Feb  3 16:26:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              35W / 250W |   1464MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--